In [11]:
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification
import torch
import numpy as np

if __name__ == '__main__':
    max_length = 256

    premise = "Two women are embracing while holding to go packages."
    hypothesis = "The men are fighting outside a deli."

    hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
    # hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
    # hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
    # hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
    # hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

    tokenizer = RobertaTokenizer.from_pretrained(hg_model_hub_name)
    model = RobertaForSequenceClassification.from_pretrained(hg_model_hub_name)

    tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)

    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

    outputs, encoding = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },
    print(outputs[0])
    predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

    print("Premise:", premise)
    print("Hypothesis:", hypothesis)
    print("Entailment:", predicted_probability[0])
    print("Neutral:", predicted_probability[1])
    print("Contradiction:", predicted_probability[2])

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-3.6270, -1.8796,  5.9538]], grad_fn=<AddmmBackward0>)
Premise: Two women are embracing while holding to go packages.
Hypothesis: The men are fighting outside a deli.
Entailment: 6.901115557411686e-05
Neutral: 0.0003960772883147001
Contradiction: 0.9995349645614624


In [12]:
encoding

tensor([[-0.2462, -0.8286, -0.1720,  ..., -0.7325,  0.6001,  0.9092]])

In [13]:
outputs, encoding = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    output_hidden_states=True,
                    labels=None)

In [14]:
torch.argmax(outputs[0]).item()

2

In [28]:
t = []
matrix = None
bias = None
dense_m = None
dense_b = None

for name, param in model.named_parameters():
    if name == 'classifier.dense.weight':
        dense_m = param
    if name == 'classifier.dense.bias':
        dense_b = param
    if name == 'classifier.out_proj.weight':
        matrix = param
    if name == 'classifier.out_proj.bias':
        bias = param

In [35]:
torch.mm(encoding, matrix.T) + bias
#model.config.id2label

tensor([[-3.6270, -1.8796,  5.9538]], grad_fn=<AddBackward0>)

In [42]:
outputs[1][0].shape

torch.Size([1, 23, 1024])

In [36]:
last_hidden = outputs[1][-1][:,0,:]
print(last_hidden.shape)
dense_d = torch.mm(last_hidden, dense_m.T) + dense_b
dense_d = torch.tanh(dense_d)
dense_d.shape
torch.mm(dense_d, matrix.T) + bias

torch.Size([1, 1024])


tensor([[-3.6270, -1.8796,  5.9538]], grad_fn=<AddBackward0>)

In [43]:
tokenizer.mask_token

'<mask>'

In [37]:
def all_masks(tokenized_text):
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    masks = [1 << i for i in range(x)]
    #     for i in range(1 << x):  # empty and full sets included here
    for i in range(1, 1 << x - 1):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        
def all_consecutive_masks(tokenized_text, max_length = -1):
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    for i in range(x):
        for j in range(i+1, x):
            mask = s[:i] + s[j:]
            if max_length > 0:
                if j - i >= max_length:
                    yield mask
            else:
                yield mask
                
def all_consecutive_masks2(tokenized_text, max_length = -1):
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    for i in range(x+1):
        for j in range(i+1, x+1):
            mask = s[i:j]
            if max_length > 0:
                if j - i <= max_length:
                    yield mask
            else:
                yield mask

def predict_json(ex, model, tokenizer):
    premise = ex['premise']
    hypothesis = ex['hypothesis']
    tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)

    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

    outputs, encoding = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    
    label = model.config.id2label[torch.argmax(outputs[0]).item()]
    
    return {'encoded_representations': encoding, 'label':label}


In [44]:

ex = {'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 
      'hypothesis': 'There is a soccer game with 12 players.', 'gold_label': 'entailment'}
    
foil = 'contradiction'#ex['gold_label']

out = predict_json(ex, model, tokenizer)
encoded_orig = out['encoded_representations']

fact = out['label']
print('Predicted: ', fact)

# assert fact != foil, "Fact should be different from the foil (if not, pick a different foil)"

ex['premise'] = ex['premise'].split()
ex['hypothesis'] = ex['hypothesis'].split()

#tokenizer.convert_tokens_to_string(out['tokens'])

masks1 = [[]]  # change this if you also want to mask out parts of the premise.
masks2 = list(all_consecutive_masks2(ex['hypothesis'], max_length=1))
encoded = []
mask_mapping = []
preds = np.zeros(shape=(len(masks1), len(masks2)))

for m1_i, m1 in enumerate(masks1):
    masked1 = list(ex['premise'])
    for i in m1:
        masked1[i] = '<mask>'
    masked1 = ' '.join(masked1)
        
    for m2_i, m2 in enumerate(masks2):
        masked2 = list(ex['hypothesis'])
        for i in m2:
            masked2[i] = '9'
        masked2 = ' '.join(masked2)
            
        masked_ex = {
            "premise": masked1,
            "hypothesis": masked2
        }
        
        masked_out = predict_json(masked_ex, model, tokenizer)
#         if masked_out['label'] != foil:
#             continue
        
        #print(m1_i, m2_i)
        #print(f"{masked1}\n{masked2}")
        print(masked_ex)
        print(masked_out['label'])
        encoded.append(masked_out['encoded_representations'].detach().numpy())
        mask_mapping.append((m1_i, m2_i))
        
        print("====")
encoded = np.array(encoded)

Predicted:  entailment
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': '9 is a soccer game with 12 players.'}
contradiction
====
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': 'There 9 a soccer game with 12 players.'}
entailment
====
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': 'There is 9 soccer game with 12 players.'}
contradiction
====
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': 'There is a 9 game with 12 players.'}
contradiction
====
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': 'There is a soccer 9 with 12 players.'}
contradiction
====
{'premise': 'A soccer game in a large area with 8 yellow players and 4 black players.', 'hypothesis': 'There is a soccer game 9 12 players.'}
entailment
====
{'premise': 'A 

In [45]:
foil = 'contradiction'
label2index = {'entailment':0, 'neutral':1, 'contradiction':2}
fact_idx = label2index[fact]
foil_idx = label2index[foil]
index2label = model.config.id2label
print('fact:', index2label[fact_idx])
print('foil:', index2label[foil_idx])
num_classifiers = 100

classifier_w = matrix.clone().detach().numpy()
classifier_b = bias.clone().detach().numpy()

u = classifier_w[fact_idx] - classifier_w[foil_idx]
contrastive_projection = np.outer(u, u) / np.dot(u, u)

print(contrastive_projection.shape)

fact: entailment
foil: contradiction
(1024, 1024)


In [47]:
from scipy.special import softmax

z_all = encoded_orig 
z_h = encoded 
z_all_row = encoded_orig @ contrastive_projection
z_h_row = encoded @ contrastive_projection

prediction_probabilities = softmax(z_all_row @ classifier_w.T + classifier_b)
prediction_probabilities = np.tile(prediction_probabilities, (z_h_row.shape[0], 1))

prediction_probabilities_del = softmax(z_h_row @ classifier_w.T + classifier_b, axis=1).squeeze(1)
print(prediction_probabilities.shape)
print(prediction_probabilities_del.shape)
p = prediction_probabilities[:, [fact_idx, foil_idx]]
q = prediction_probabilities_del[:, [fact_idx, foil_idx]]

p = p / p.sum(axis=1).reshape(-1, 1)
q = q / q.sum(axis=1).reshape(-1, 1)
distances = (p[:, 0] - q[:, 0])

print(' '.join(ex['premise']))
print(' '.join(ex['hypothesis']))

print("=========\n=======Farthest masks:=======")    
    
highlight_rankings = np.argsort(-distances)

for i in range(4):
    rank = highlight_rankings[i]
    m1_i, m2_i = mask_mapping[rank]
    
    masked1 = list(ex['premise'])
    for k in masks1[m1_i]:
        masked1[k] = '<mask>'
    masked1 = ' '.join(masked1)
    
    masked2 = list(ex['hypothesis'])
    for k in masks2[m2_i]:
        masked2[k] = '<mask>'
    masked2 = ' '.join(masked2)
    
    print(masked1)
    print(masked2)
    print(np.round(distances[rank], 4))

(8, 3)
(8, 3)
A soccer game in a large area with 8 yellow players and 4 black players.
There is a soccer game with 12 players.
=======Farthest masks:=======
A soccer game in a large area with 8 yellow players and 4 black players.
9 is a soccer game with 12 players.
0.3406
A soccer game in a large area with 8 yellow players and 4 black players.
There 9 a soccer game with 12 players.
0.3406
A soccer game in a large area with 8 yellow players and 4 black players.
There is 9 soccer game with 12 players.
0.3406
A soccer game in a large area with 8 yellow players and 4 black players.
There is a 9 game with 12 players.
0.3406
